In [3]:
import pandas as pd
import plotly.graph_objects as go

OUT_DIR = "./spillover_outputs"
GIRF_PATH = f"{OUT_DIR}/girf_long.csv"

girf = pd.read_csv(GIRF_PATH)
# columns: horizon, response, shock, girf

girf["horizon"] = girf["horizon"].astype(int)

shocks = sorted(girf["shock"].unique().tolist())
responses = sorted(girf["response"].unique().tolist())

# Create one figure with dropdown to select shock
fig = go.Figure()

# Add traces for the FIRST shock initially (one trace per response)
init_shock = shocks[0]
for resp in responses:
    d = girf[(girf["shock"] == init_shock) & (girf["response"] == resp)].sort_values("horizon")
    fig.add_trace(go.Scatter(
        x=d["horizon"],
        y=d["girf"],
        mode="lines",
        name=resp,
        visible=True
    ))

# For each other shock, add a full set of response traces but set visible=False
# We'll toggle visibility in dropdown.
trace_blocks = {init_shock: list(range(len(responses)))}
base = len(responses)

for s in shocks[1:]:
    idxs = []
    for resp in responses:
        d = girf[(girf["shock"] == s) & (girf["response"] == resp)].sort_values("horizon")
        fig.add_trace(go.Scatter(
            x=d["horizon"],
            y=d["girf"],
            mode="lines",
            name=resp,
            visible=False
        ))
        idxs.append(base)
        base += 1
    trace_blocks[s] = idxs

# Build dropdown buttons
buttons = []
total_traces = len(fig.data)

for s in shocks:
    vis = [False] * total_traces
    for idx in trace_blocks[s]:
        vis[idx] = True
    buttons.append(dict(
        label=s,
        method="update",
        args=[
            {"visible": vis},
            {"title": f"GIRF (Generalized IRF) — shock: {s}", "legend": {"title": "Response"}}
        ]
    ))

# Horizontal zero line (shape)
fig.update_layout(
    title=f"GIRF (Generalized IRF) — shock: {init_shock}",
    xaxis_title="Horizon (days)",
    yaxis_title="Response",
    hovermode="x unified",
    height=650,
    updatemenus=[dict(
        buttons=buttons,
        direction="down",
        x=1.02, xanchor="left",
        y=0.5, yanchor="top"
    )],
    shapes=[dict(
        type="line",
        x0=0, x1=int(girf["horizon"].max()),
        y0=0, y1=0,
        xref="x", yref="y"
    )],
    margin=dict(r=220)  # space for dropdown
)

fig.show()
